In [ ]:
import pygplates
import pygmt
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter1d
#from sklearn.neighbors import KernelDensity
from gprm import PointDistributionOnSphere
from gprm.datasets import Reconstructions, Zircons, Rocks
from gprm.utils.sphere import spherical_kde
from gprm.utils.raster import to_anchor_plate
from gprm import SubductionConvergence
import warnings

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import matplotlib as mpl
mpl.rc('font',family='Helvetica')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2

%load_ext autoreload
%autoreload 2


In [ ]:
df_SampleDetails, df_Data = Zircons.loadDB(version=2018)


In [ ]:
SedimentaryZircons = Zircons.get_sedimentary_samples(df_SampleDetails,df_Data)
IgneousZircons = Zircons.get_igneous_samples(df_SampleDetails,df_Data)

In [ ]:
sam_mask = gpd.read_file('/Users/simon/GIT/NREE/gis/SAM_zircon_mask.geojson')

sam_Izircons = gpd.clip(IgneousZircons, sam_mask)
sam_Szircons = gpd.clip(SedimentaryZircons, sam_mask)

In [ ]:
#for reconstruction_time in time_series:
#    subset = IZM[(IZM['206Pb_238U_Age_Ma']>=reconstruction_time) & (IZM['206Pb_238U_Age_Ma']<reconstruction_time+time_step)]
 
sam_Izircons['206Pb_238U_Age_Ma'].hist(bins=np.arange(0,600,5))    

In [ ]:
M2022 = Reconstructions.fetch_Muller2022()
M2022_NNR = Reconstructions.fetch_Muller2022(NNR=True)
M2022_NNR.name = M2022_NNR.name+'_NNR'
M2016 = Reconstructions.fetch_Matthews2016()
M2019 = Reconstructions.fetch_Muller2019()

Cupor = gpd.clip(Rocks.BaseMetalDeposits(deposit_type='Cu-por'), sam_mask)
Cupor = Cupor.replace({'ND': np.nan})

#reconstruction_model = M2019

def calculate_andes_kinematics(reconstruction_model, time_series = np.arange(0,240,2), lat_samples = np.arange(-55,5,0.1)):
    
    anchor_plate_id = 201

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sc_df = SubductionConvergence(reconstruction_model, time_series, np.radians(0.1), anchor_plate_id=anchor_plate_id, velocity_delta_time=1).df
        cr = np.asarray(sc_df['conv_rate'])
        co = np.asarray(sc_df['conv_obliq'])
        sc_df['ortho_conv_rate'] = pd.Series(cr*np.abs(np.cos(np.radians(co))), index=sc_df.index)
        sc_df['sub_area'] = sc_df['ortho_conv_rate'] * 100. * sc_df['arc_length'] * 2 * np.pi * 6371000. / 360.


    anchor_plate_id = 0

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sc_tmp = SubductionConvergence(reconstruction_model, time_series, np.radians(0.1), anchor_plate_id=anchor_plate_id, velocity_delta_time=1).df
        mr = np.asarray(sc_tmp['migr_rate'])
        mo = np.asarray(sc_tmp['migr_obliq'])    
        sc_tmp['ortho_migr_rate'] = pd.Series(mr*np.sin(np.radians(np.abs(mo))), index=sc_tmp.index)


    scc = sc_df[(sc_df.subducting_plate.isin([902,908,919,904,911,985,224,802])) & 
                ((sc_df.overriding_plate.isin([201,701,291])) | (sc_df.overriding_plate>200000))]
    sccM = sc_tmp[(sc_tmp.subducting_plate.isin([902,908,919,904,911,985,224,802])) & 
                  ((sc_tmp.overriding_plate.isin([201,701,291])) | (sc_tmp.overriding_plate>200000))]

    scc = scc.reset_index()
    sccM = sccM.reset_index()
    print(len(scc), len(sccM))
    scc['migr_rate'] = sccM['migr_rate']
    scc['migr_obliq'] = sccM['migr_obliq']
    scc['ortho_migr_rate'] = sccM['ortho_migr_rate']

    scc = scc[(scc.lat<15) & (scc.lon<0)]
    
    return scc


def nice_axes(ax, cm, label, ylabel=False, xlabel=False):
    ax.set_xlim(340,0)
    ax.set_ylim(-50,10)
    cax = inset_axes(ax, width="20%", height="4%", loc='upper left', borderpad=1.)
    cbar = plt.colorbar(cm, cax=cax, orientation='horizontal', extend='both')
    cax.xaxis.set_ticks_position("bottom")
    cbar.ax.set_xlabel(label)
    if ylabel:
        ax.set_ylabel('Latitude [deg.]')
    if xlabel:
        ax.set_xlabel('Reconstruction Age [Myr]')
          
            
def plot_andes_kinematics(reconstruction_model, scc, time_series, lat_samples = np.arange(-55,5,0.1)):

    migr_rate_array = []
    ortho_migr_rate_array = []
    migr_obliq_array = []
    conv_rate_array = []
    ortho_conv_rate_array = []
    conv_obliq_array = []

    for reconstruction_time in time_series:

        #print(reconstruction_time)
        subset = scc[scc.time==reconstruction_time]
        subset = subset.sort_values(by='lat').reset_index()

        zi = np.interp(lat_samples, subset.lat, subset.migr_rate, left=np.nan, right=np.nan)
        migr_rate_array.append(zi)
        zi = np.interp(lat_samples, subset.lat, subset.ortho_migr_rate, left=np.nan, right=np.nan)
        ortho_migr_rate_array.append(zi)
        zi = np.interp(lat_samples, subset.lat, subset.migr_obliq, left=np.nan, right=np.nan)
        migr_obliq_array.append(zi)

        zi = np.interp(lat_samples, subset.lat, subset.conv_rate, left=np.nan, right=np.nan)
        conv_rate_array.append(zi)
        zi = np.interp(lat_samples, subset.lat, subset.ortho_conv_rate, left=np.nan, right=np.nan)
        ortho_conv_rate_array.append(zi)
        zi = np.interp(lat_samples, subset.lat, subset.conv_obliq, left=np.nan, right=np.nan)
        conv_obliq_array.append(zi)


    migr_rate_array = np.vstack(migr_rate_array).T
    ortho_migr_rate_array = np.vstack(ortho_migr_rate_array).T
    migr_obliq_array = np.vstack(migr_obliq_array).T
    conv_rate_array = np.vstack(conv_rate_array).T
    ortho_conv_rate_array = np.vstack(ortho_conv_rate_array).T
    conv_obliq_array = np.vstack(conv_obliq_array).T


    fig = plt.figure(figsize=(14,10))
    gs = fig.add_gridspec(nrows=3, ncols=2, wspace=0.1, hspace=.15)

    ax = fig.add_subplot(gs[0, 0])
    cm = ax.pcolormesh(time_series, 
                   lat_samples, 
                   conv_rate_array, 
                   vmin=0, vmax=15, cmap='Oranges')
    ax.plot(Cupor.Age, Cupor.geometry.y, 'ko', alpha=0.5)
    nice_axes(ax, cm, 'full\nconvergence rate\n[mm/yr]', ylabel=True)

    ax = fig.add_subplot(gs[1, 0])
    cm = ax.pcolormesh(time_series, 
                   lat_samples, 
                   ortho_conv_rate_array, 
                   vmin=0, vmax=15, cmap='Oranges')
    ax.plot(Cupor.Age, Cupor.geometry.y, 'ko', alpha=0.5)
    nice_axes(ax, cm, 'trench-orthogonal\nconvergence rate\n[mm/yr]', ylabel=True)

    ax = fig.add_subplot(gs[2, 0])
    cm = ax.pcolormesh(time_series, 
                   lat_samples, 
                   conv_obliq_array, 
                   vmin=-180, vmax=180, cmap='twilight_shifted', alpha=0.7)
    ax.plot(Cupor.Age, Cupor.geometry.y, 'ko', alpha=0.5)
    nice_axes(ax, cm, 'convergence\nobliquity\n[deg]', xlabel=True, ylabel=True)

    ax = fig.add_subplot(gs[0, 1])
    cm = ax.pcolormesh(time_series, 
                   lat_samples, 
                   migr_rate_array, 
                   vmin=-10, vmax=10, cmap='seismic_r')
    ax.plot(Cupor.Age, Cupor.geometry.y, 'ko', alpha=0.5)
    nice_axes(ax, cm, 'full\nmigration rate\n[mm/yr]')

    ax = fig.add_subplot(gs[1, 1])
    cm = ax.pcolormesh(time_series, 
                   lat_samples, 
                   ortho_migr_rate_array, 
                   vmin=-10, vmax=10, cmap='seismic_r')
    ax.plot(Cupor.Age, Cupor.geometry.y, 'ko', alpha=0.5)
    nice_axes(ax, cm, 'trench-orthogonal\nmigration rate\n[mm/yr]')

    ax = fig.add_subplot(gs[2, 1])
    cm = ax.pcolormesh(time_series, 
                   lat_samples, 
                   migr_obliq_array, 
                   vmin=-180, vmax=180, cmap='twilight_shifted', alpha=0.7)
    ax.plot(Cupor.Age, Cupor.geometry.y, 'ko', alpha=0.5)
    nice_axes(ax, cm, 'migration\nobliquity\n[deg]', xlabel=True)

    plt.savefig('../images/Plate_Kinematics_Andes_{:s}.png'.format(reconstruction_model.name))
    plt.show()

In [ ]:
for reconstruction_model in [M2022, M2022_NNR, M2016, M2019]:
    
    print(reconstruction_model.name)
    if 'Muller++2019' in reconstruction_model.name:
        print('here')
        time_series = np.arange(0,240,2)
    else:
        time_series = np.arange(0,340,2)
    
    scc = calculate_andes_kinematics(reconstruction_model, time_series)
    
    plot_andes_kinematics(reconstruction_model, scc, time_series)
    
    

In [ ]:
import xycmap

corner_colors = ("red", "orange", "dodgerblue", "white")
n = (6, 6)  # x, y
cmap = xycmap.custom_xycmap(corner_colors=corner_colors, n=n)


conv_color_input = migr_rate_array.flatten()
conv_color_input[np.isnan(conv_color_input)] = 0
age_color_input = migr_obliq_array.flatten()
age_color_input[np.isnan(age_color_input)] = 0
colors = xycmap.bivariate_color(sx=conv_color_input, sy=age_color_input, cmap=cmap,
                                xlims=(-10,10), ylims=(-180,180))

tmp = np.reshape(np.vstack(colors),(migr_rate_array.shape[0],migr_rate_array.shape[1],4))

fig,ax = plt.subplots(figsize=(15,7))
ax.imshow(tmp, extent=[time_series.min(), time_series.max(), lat_samples.min(), lat_samples.max()], origin='lower', aspect='auto')
cax = fig.add_axes([0.9, 0.25, 0.5, 0.5])
cax = xycmap.bivariate_legend(ax=cax, sx=conv_color_input, sy=age_color_input, cmap=cmap,
                              xlims=(-10,10), ylims=(-180,180))
